In [ ]:
import pandas as pd
print(pd.__version__)
from utils.utils import *

from utils.anthropic_utils import *
from utils.perturbation_utils import *

# Get pertubations

In [ ]:
strings = generate_tir_dict_severehypo()

In [ ]:
prompt_type = 'Metric'

name = "Batch_perturbation_severe"
dataset = 'CSII'
model ='claude-3-haiku-20240307'

batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"
os.makedirs(batch_dir, exist_ok=True)

individual_data = load_data_metrics_for_prompt_short(dataset) 

In [ ]:
prompt_list = {}
for (hypo, hyper,tir), description in strings.items():
    for id in individual_data:

        pat = individual_data[id]
        comparator = description

        prompt_list[(hypo, hyper,tir,id)] = instructions(comparator,pat)

In [ ]:
i=0
save_dict = {}
for (hypo, hyper,tir,id), prompt in prompt_list.items():
    save_dict[i] = {
        "hypo": hypo,
        "hyper": hyper,
        "tir": tir,
        "id": id,
        "prompt": prompt
    }
    i+=1


In [ ]:

model = "claude-3-7-sonnet-20250219"
model ="claude-3-haiku-20240307"
os.makedirs(batch_dir, exist_ok=True)

batch_dir =f"{name}/{dataset}/{prompt_type}/{model}/"

batch_json, requests = make_batch_json_anthropic_perturbation(prompt_list, batch_dir , model)

save_json(
    data=requests,
    directory=batch_dir,
    custom_ending="requests.jsonl"
)
simple_data = {
        'batch_id': batch_json.id,
        'status': batch_json.processing_status,
        'created_at': batch_json.created_at.isoformat() if batch_json.created_at else None,
        'saved_at': datetime.now().isoformat()
    }
save_json(
        data=simple_data,
        directory=batch_dir,
        custom_ending="batch_sent.jsonl")
    

